# Model Training and Comparison

In [10]:
import sys
import os
import time
import torch
import wandb
from datetime import datetime

sys.path.append('../')
from src.core.trainer import DialogTrainer
from src.config.settings import get_model_config, get_test_config, get_development_config
from src.data.loaders import get_dataset_manager

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0xf949465ae9e0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at f948af0a2cb0, raw_cell="import sys
import os
import time
import torch
impo.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu-22.04/home/khalil/DNN_tmp/llm/notebooks/02_model_training.ipynb#Y244sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Using device: cpu
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0xf949465ae9e0>> (for post_run_cell), with arguments args (<ExecutionResult object at f948af0a2fe0, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at f948af0a2cb0, raw_cell="import sys
import os
import time
import torch
impo.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu-22.04/home/khalil/DNN_tmp/llm/notebooks/02_model_training.ipynb#Y244sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
# Clean up any existing wandb runs
try:
    wandb.finish()
    print("Cleaned up existing wandb run")
except:
    print("No existing wandb run to clean up")

# Setup WandB authentication
from dotenv import load_dotenv

load_dotenv()
wandb_api_key = os.getenv('WANDB_API_KEY')

if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("✓ WandB connection established")
else:
    print("⚠ No WandB key found - need to fix the .env file")
    print("Reminder: Add WANDB_API_KEY=your_actual_key to .env file")
    print("Training will continue without WandB logging")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0xf949465ae9e0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at f948af7332b0, raw_cell="# Setup WandB authentication
from dotenv import lo.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu-22.04/home/khalil/DNN_tmp/llm/notebooks/02_model_training.ipynb#Y245sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


✓ WandB connection established
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0xf949465ae9e0>> (for post_run_cell), with arguments args (<ExecutionResult object at f948af733400, execution_count=11 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at f948af7332b0, raw_cell="# Setup WandB authentication
from dotenv import lo.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu-22.04/home/khalil/DNN_tmp/llm/notebooks/02_model_training.ipynb#Y245sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [19]:
# Initialize main WandB run for model comparison experiment
main_run = wandb.init(
    project="dialog-model-training",
    name=f"model-comparison-{datetime.now().strftime('%Y%m%d-%H%M')}",
    tags=["comparison", "training", "notebook"],
    notes="Comparing multiple dialog model architectures: GPT-2, DialoGPT, and custom models"
)

print("✓ Main WandB experiment tracking started")
print(f"Run name: {main_run.name}")
print(f"Project: {main_run.project}")

✓ Main WandB experiment tracking started
Run name: model-comparison-20250722-1722
Project: dialog-model-training


In [ ]:
# Load dataset and configuration
dataset_manager = get_dataset_manager()
df = dataset_manager.load_dataset()
training_config = get_development_config()

# Models to compare
models_to_train = [
    #"gpt2-small",      # Pre-trained GPT-2
    "custom-small",    # Custom model from scratch
    #"dialogpt-small"   # Pre-trained DialoGPT
]

# Single consolidated output to avoid duplicates
output = []
output.append("Training Configuration:")
output.append(f"- Epochs: {training_config.num_epochs}")
output.append(f"- Batch size: {training_config.batch_size}")
output.append(f"- Max samples: {training_config.max_samples}")
output.append(f"- Learning rate: {training_config.learning_rate}")
output.append(f"\nModels to train and compare: {models_to_train}")
output.append(f"Dataset loaded: {len(df)} samples")

print("\n".join(output))

Loading dataset from local file: data/alpaca-gpt4.csv
Training Configuration:
- Epochs: 1
- Batch size: 2
- Max samples: 100
- Learning rate: 5e-05

Models to train and compare: ['custom-small']
Dataset loaded: 52002 samples
Training Configuration:
- Epochs: 1
- Batch size: 2
- Max samples: 100
- Learning rate: 5e-05

Models to train and compare: ['custom-small']
Dataset loaded: 52002 samples


## Training Loop with WandB Tracking

In [ ]:
def train_model_with_logging(model_name, df, training_config, main_wandb_run):
    """Train a model with streamlined WandB logging"""
    
    # Get model config and initialize trainer with wandb run
    model_config = get_model_config(model_name)
    trainer = DialogTrainer(model_config=model_config, wandb_run=main_wandb_run)
    
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")
    
    # Prepare dataset
    train_dataset, eval_dataset = trainer.prepare_dataset(
        df, 
        max_samples=training_config.max_samples
    )
    
    # Start training with wandb logging built into trainer
    start_time = datetime.now()
    trainer.train(
        train_dataset, 
        eval_dataset, 
        num_epochs=training_config.num_epochs,
        batch_size=training_config.batch_size,
        learning_rate=training_config.learning_rate
    )
    end_time = datetime.now()
    
    training_time = (end_time - start_time).total_seconds()
    print(f"Training completed in {training_time/60:.1f} minutes")
    
    return trainer

# Store trained models for comparison
trained_models = {}

In [ ]:
# Train all models using the main wandb run
for model_name in models_to_train:
    print(f"\nStarting training for {model_name}...")
    try:
        trainer = train_model_with_logging(model_name, df, training_config, main_run)
        trained_models[model_name] = trainer
        print(f"SUCCESSFULLY trained {model_name}")
    except Exception as e:
        print(f"FAILED to train {model_name}: {e}")
        continue

print(f"\nTraining Summary:")
print(f"Successfully trained: {list(trained_models.keys())}")
print(f"Total models trained: {len(trained_models)}")

# Log training completion summary to main run
main_run.log({
    "training/completed": True,
    "training/models_successfully_trained": len(trained_models),
    "training/completion_time": datetime.now().timestamp(),
    "training/successful_models": list(trained_models.keys())
})

Closing initial WandB run...



Starting training for custom-small...


Building custom model from scratch: custom-gpt-small
Architecture: 8 layers, 512 dim, 8 heads
Output directory: ./models/custom_small
Custom GPT model initialized:
  Parameters: 76,945,408
  Layers: 8
  Embedding dim: 512
  Attention heads: 8
Device: cpu
Model parameters: 76,945,408

Training custom-small
Using 100 samples for training
Training samples: 90
Evaluation samples: 10
Custom GPT model initialized:
  Parameters: 76,945,408
  Layers: 8
  Embedding dim: 512
  Attention heads: 8
Device: cpu
Model parameters: 76,945,408

Training custom-small
Using 100 samples for training
Training samples: 90
Evaluation samples: 10


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

...

/home/khalil/DNN_tmp/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


Training completed! Model saved to ./models/custom_small
Training completed in 0.8 minutes


dataset_size,▁
eval_size,▁
model_parameters,▁
train/epoch,▁
train/global_step,▁▁
training_time_minutes,▁
training_time_seconds,▁
dataset_size,90
eval_size,10
model_parameters,76945408
total_flos,6872362598400.0


SUCCESSFULLY trained custom-small

Training Summary:
Successfully trained: ['custom-small']
Total models trained: 1

Initializing main WandB run for generation testing...


## Dialog Generation Testing

In [ ]:
# Test instructions for dialog generation
test_instructions = [
    "Explain what machine learning is in simple terms.",
    "How do I make a good cup of coffee?",
    "What are the benefits of exercise?",
    "Tell me about the solar system.",
    "How can I improve my communication skills?"
]

print("Dialog Generation Comparison")
print("=" * 60)

# Create wandb table for generation results
generation_table = wandb.Table(columns=[
    "Model", "Instruction", "Response", "Response_Length", 
    "Word_Count", "Generation_Time_Seconds", "Instruction_ID"
])

# Store metrics for aggregation
generation_metrics = {}

for i, instruction in enumerate(test_instructions, 1):
    print(f"\n[{i}/{len(test_instructions)}] Instruction: {instruction}")
    print("-" * 40)
    
    for model_name, trainer in trained_models.items():
        try:
            start_time = datetime.now()
            # Use wandb logging in generation
            response = trainer.generate_response(instruction, max_length=100, log_to_wandb=True)
            generation_time = (datetime.now() - start_time).total_seconds()
            
            print(f"\n{model_name}: {response}")
            
            # Add to wandb table
            generation_table.add_data(
                model_name,
                instruction[:100] + "..." if len(instruction) > 100 else instruction,
                response[:200] + "..." if len(response) > 200 else response,
                len(response),
                len(response.split()),
                round(generation_time, 3),
                i
            )
            
            # Track metrics per model for aggregation
            if model_name not in generation_metrics:
                generation_metrics[model_name] = {
                    "total_time": 0,
                    "total_responses": 0,
                    "total_length": 0,
                    "total_words": 0,
                    "successful_generations": 0
                }
            
            generation_metrics[model_name]["total_time"] += generation_time
            generation_metrics[model_name]["total_responses"] += 1
            generation_metrics[model_name]["total_length"] += len(response)
            generation_metrics[model_name]["total_words"] += len(response.split())
            generation_metrics[model_name]["successful_generations"] += 1
            
        except Exception as e:
            print(f"\n{model_name}: Error - {e}")
            # Log error to wandb
            main_run.log({
                f"generation/error_{model_name}_instruction_{i}": str(e)
            })
    
    print("=" * 60)

print(f"Generation testing completed!")
print(f"Generated {sum(metrics['successful_generations'] for metrics in generation_metrics.values())} total responses")

Dialog Generation Comparison

[1/5] Instruction: Explain what machine learning is in simple terms.
----------------------------------------

custom-small: bill





.


 the.


.

.







 the
 options the




,
 the



 the


 task the
. Store,bill such Quan
 the


 helmet
,. the
, a,

xton.,

 the


.
,,
 Quan

[2/5] Instruction: How do I make a good cup of coffee?
----------------------------------------

custom-small: bill





.


 the.


.

.







 the
 options the




,
 the



 the


 task the
. Store,bill such Quan
 the


 helmet
,. the
, a,

xton.,

 the


.
,,
 Quan

[2/5] Instruction: How do I make a good cup of coffee?
----------------------------------------

custom-small: the.

,








,
 the





 the


bill





 reliable
 the Thirty thexton






 options925
 final and

 the such the


 Stanford.






.




,
,
,


 Columbia,.dm.

,
 the horse and
 horse. legislative

[3/5] Instruction: What are the benefits of exercise?
----------------------------------------


In [ ]:
# Log the comprehensive generation comparison table
main_run.log({"generation_comparison_detailed": generation_table})

# Log aggregated metrics for each model
print("Logging final generation metrics to WandB...")

for model_name, metrics in generation_metrics.items():
    if metrics["total_responses"] > 0:  # Avoid division by zero
        avg_metrics = {
            f"generation_summary/{model_name}_avg_time": metrics["total_time"] / metrics["total_responses"],
            f"generation_summary/{model_name}_avg_length": metrics["total_length"] / metrics["total_responses"],
            f"generation_summary/{model_name}_avg_words": metrics["total_words"] / metrics["total_responses"],
            f"generation_summary/{model_name}_total_time": metrics["total_time"],
            f"generation_summary/{model_name}_success_rate": metrics["successful_generations"] / len(test_instructions),
            f"generation_summary/{model_name}_responses_generated": metrics["successful_generations"]
        }
        main_run.log(avg_metrics)

# Create a model comparison summary table
comparison_summary = wandb.Table(columns=[
    "Model", "Avg_Generation_Time", "Avg_Response_Length", 
    "Avg_Word_Count", "Success_Rate", "Total_Responses"
])

for model_name, metrics in generation_metrics.items():
    if metrics["total_responses"] > 0:
        comparison_summary.add_data(
            model_name,
            round(metrics["total_time"] / metrics["total_responses"], 3),
            round(metrics["total_length"] / metrics["total_responses"], 1),
            round(metrics["total_words"] / metrics["total_responses"], 1),
            round(metrics["successful_generations"] / len(test_instructions), 2),
            metrics["successful_generations"]
        )

main_run.log({"model_comparison_summary": comparison_summary})

# Log experiment completion
final_summary = {
    "experiment/status": "completed",
    "experiment/models_trained": len(trained_models),
    "experiment/test_instructions": len(test_instructions),
    "experiment/total_generations": sum(metrics["successful_generations"] for metrics in generation_metrics.values()),
    "experiment/completion_timestamp": datetime.now().isoformat(),
}

main_run.log(final_summary)

# Close WandB run
main_run.finish()

print("✓ Experiment completed successfully")
print(f"✓ Trained {len(trained_models)} models")
print(f"✓ Generated {sum(metrics['successful_generations'] for metrics in generation_metrics.values())} responses")
print("✓ All results logged to WandB")

Logging generation metrics to WandB...


custom-small_avg_generation_time,▁
custom-small_avg_response_length,▁
custom-small_avg_response_words,▁
custom-small_responses_generated,▁
custom-small_total_generation_time,▁
models_successfully_trained,▁
models_trained,▁
total_generations,▁
total_test_instructions,▁
training_completion_time,▁
custom-small_avg_generation_time,7.95595


Experiment completed successfully.
Trained 1 models and completed generation testing.
